In [1]:
import os
import json
import time
import collections
import pandas as pd
# 禁用科学计数法
pd.set_option('display.float_format',lambda x : '%.2f' % x)

In [2]:
def convert_time(time_str):
    """
    input example: 2020/12/21 8:09:14
    """
    time_str = time_str.strip()
    ymd, hmis = time_str.split(' ')
    y, m, d = ymd.split('/')
    y, m, d = int(y), int(m), int(d)
    h, mi, s = hmis.split(':')
    h, mi, s = int(h), int(mi), int(float(s))
    time_tuple = (y, m, d, h, mi, s, 0, 0, 0)
    return time.mktime(time_tuple)

In [3]:
point_dict = collections.defaultdict(set)
fpath = '../data/'
files = os.listdir(fpath)
track_data = []
for f in files:
    if f.startswith('gxdc_gj') and f.endswith('.csv'):
        track_data.append(f)
for fname in track_data:
    day = fname[fname.find('2020') + len('2020'):fname.find('.csv')]
    dd = day[2:]
    df = pd.read_csv(fpath + fname)
    era_start = time.mktime((2020, 12, int(dd), 6, 0, 0, 0, 0, 0))
    era_end = time.mktime((2020, 12, int(dd), 10, 0, 0, 0, 0, 0))
    df['MKTIME'] = df['LOCATING_TIME'].apply(convert_time)
    df.sort_values("MKTIME", inplace = True)
    df.reset_index(drop = True, inplace = True)
    curidx = 0
    cur_timerange = era_start
    curkey = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(cur_timerange))
    while curidx < len(df):
        if df['MKTIME'][curidx] > cur_timerange + 1800:
            # 30 min 一个区间,key是该区间起始时间
            cur_timerange += 1800
            curkey = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(cur_timerange))
        point_dict[curkey].add((df['LATITUDE'][curidx], df['LONGITUDE'][curidx]))
        curidx += 1
    for k in point_dict:
        point_dict[k] = list(point_dict[k])
    

In [4]:
f = open("../cleaned_data/bike_tracks.json","w")
f.write(json.dumps(point_dict))
f.close()